# Experiments with images 


IMPORTANT!- Change local path to images wherever there is "image=..." in the code

Main questions: Which of the methods (brightness, contrast, color correction, saturation, hue, exposure), etc is best to use? Should we use a combination? 

Run this cell first to import cv2 and to check if your image loads correctly. 

In [1]:
import cv2

# Escape each backslash with another backslash
image = cv2.imread('C:\\Users\\User\\OneDrive\\Documents\\Data Challenge 3\\DC-3-Group-12\\Preprocessing\\9870_Gerres_f000170.jpg')

if image is None:
    print("Error: Could not load image. Check the file path.")
else:
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

Error: Could not load image. Check the file path.


In [2]:
import cv2

# Specify the path to your image
image_path = r'C:\Users\User\OneDrive\Documents\Data Challenge 3\DC-3-Group-12\Preprocessing\9908_Epinephelus_f000127.jpg'

# Load the image
image = cv2.imread(image_path)

# Display the image
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


This code just adds contrast and brightness

In [11]:
alpha = 1.5  # Contrast control (1.0-3.0), >1 increases contrast, <1 decreases contrast
beta = 0   # Brightness control (0-100), >0 increases brightness, <0 decreases brightness

# Apply the adjustments to the image
adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

# Display the original and adjusted images side by side
cv2.imshow('Original Image', image)
cv2.imshow('Adjusted Brightness and Contrast Image', adjusted_image)

# Wait for a key press and close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()

This code first checks the brightness and the contrast of the image and enhances them as needed based on a certain threshold. 

In [9]:
import cv2
import numpy as np

def calculate_brightness_contrast(image):
    # Convert image to grayscale for brightness calculation
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate the mean brightness
    brightness = np.mean(gray)

    # Calculate contrast as the standard deviation of pixel values
    contrast = np.std(gray)

    return brightness, contrast

def adjust_brightness_contrast(image, target_brightness=128, target_contrast=64):
    brightness, contrast = calculate_brightness_contrast(image)
    
    # Calculate the adjustment factors, but constrain them to avoid extreme adjustments
    alpha = min(max(target_contrast / max(contrast, 1e-6), 0.5), 1.5)  # Adjust contrast, constrained between 0.5 and 1.5
    beta = min(max(target_brightness - brightness, -100), 100)  # Adjust brightness, constrained between -100 and 100
    
    # Apply the adjustments
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image
     
# Load the image
image_path = r'C:\Users\User\OneDrive\Documents\Data Challenge 3\DC-3-Group-12\Preprocessing\7393_F1_f000513.jpg'
image = cv2.imread(image_path)

if image is None:
    print("Error: Could not load image. Check the file path.")
else:
    # Calculate initial brightness and contrast
    initial_brightness, initial_contrast = calculate_brightness_contrast(image)
    print(f"Initial Brightness: {initial_brightness:.2f}")
    print(f"Initial Contrast: {initial_contrast:.2f}")

    # Adjust brightness and contrast based on initial values
    adjusted_image = adjust_brightness_contrast(image)

    # Calculate adjusted brightness and contrast
    adjusted_brightness, adjusted_contrast = calculate_brightness_contrast(adjusted_image)
    print(f"Adjusted Brightness: {adjusted_brightness:.2f}")
    print(f"Adjusted Contrast: {adjusted_contrast:.2f}")

    # Display the original and adjusted images
    cv2.imshow('Original Image', image)
    cv2.imshow('Adjusted Image', adjusted_image)

    # Wait for a key press and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Initial Brightness: 145.08
Initial Contrast: 40.04
Adjusted Brightness: 193.00
Adjusted Contrast: 49.74


This code is a combination of methods: adjust brightness, contrast, does Color Correction with 
White Balance Adjustment: Underwater images often have a blue or green tint. Correcting white balance helps to restore natural colors.
Histogram Equalization: Equalize the histogram of each color channel (R, G, B) to balance color distribution. Also it applie Bilateral Filter: Smoothens the image while preserving edges, which is useful for reducing noise. It also adds . CLAHE (Contrast Limited Adaptive Histogram Equalization)
CLAHE enhances contrast in local regions of the image, improving details without over-amplifying noise. However using them all together is probably too much. 

In [19]:
import cv2
import numpy as np

def calculate_brightness_contrast(image):
    # Convert image to grayscale for brightness calculation
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate the mean brightness and contrast
    brightness = np.mean(gray)
    contrast = np.std(gray)
    return brightness, contrast

def adjust_brightness_contrast(image, alpha=1.5, beta=30):
    # Adjust brightness and contrast
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def correct_white_balance(image):
    # Convert to LAB color space and apply CLAHE
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    corrected_image = cv2.merge((l, a, b))
    return cv2.cvtColor(corrected_image, cv2.COLOR_LAB2BGR)

def apply_bilateral_filter(image):
    # Apply bilateral filter for smoothing
    return cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

def apply_clahe(image):
    # Apply CLAHE to enhance local contrast
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    enhanced_image = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(enhanced_image, cv2.COLOR_LAB2BGR)

# Load the underwater image
image_path = r'C:\Users\User\OneDrive\Documents\Data Challenge 3\DC-3-Group-12\Preprocessing\7393_F1_f000513.jpg'
image = cv2.imread(image_path)

if image is None:
    print("Error: Could not load image. Check the file path.")
else:
    # Adjust brightness and contrast
    adjusted_image = adjust_brightness_contrast(image)

    # Apply white balance correction
    wb_corrected_image = correct_white_balance(adjusted_image)

    # Apply CLAHE
    clahe_image = apply_clahe(wb_corrected_image)

    # Apply bilateral filter
    final_image = apply_bilateral_filter(clahe_image)

    # Display the original and enhanced images
    cv2.imshow('Original Image', image)
    cv2.imshow('Enhanced Image', final_image)

    # Wait for a key press and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()


This code enhances saturation. Saturation modifies the color intensity. Larger values apply greater variance. 

In [5]:
import cv2
import numpy as np

def enhance_saturation(image, saturation_scale=1.5):
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Split the channels
    h, s, v = cv2.split(hsv_image)
    
    # Scale the saturation channel
    s = s.astype(np.float32)  # Convert to float to prevent overflow
    s *= saturation_scale
    
    # Clip the values to ensure they stay within [0, 255] and convert back to uint8
    s = np.clip(s, 0, 255).astype(np.uint8)
    
    # Merge the channels back
    enhanced_hsv = cv2.merge([h, s, v])
    
    # Convert back to BGR color space
    enhanced_image = cv2.cvtColor(enhanced_hsv, cv2.COLOR_HSV2BGR)
    return enhanced_image

# Load the image
image_path = r'C:\Users\User\OneDrive\Documents\Data Challenge 3\DC-3-Group-12\Preprocessing\7393_F1_f000513.jpg'
image = cv2.imread(image_path)

if image is None:
    print("Error: Could not load image. Check the file path.")
else:
    # Enhance the saturation of the image
    enhanced_image = enhance_saturation(image, saturation_scale=1.5)

    # Display the original and enhanced images
    cv2.imshow('Original Image', image)
    cv2.imshow('Enhanced Saturation Image', enhanced_image)

    # Wait for a key press and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()

This code enhances exposure. (Exposure is kinda like a combination of brightness and contrast). Exposure determines the amount of black or white that is added to colors. The higher the value, the greater the variance, possibly making it appear as if the images were over-or under-exposed.

In [6]:
import cv2
import numpy as np

def enhance_exposure(image, exposure_factor=1.2):
    # Convert the image to the LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    
    # Split the LAB image into separate channels
    l, a, b = cv2.split(lab_image)
    
    # Convert the L channel to float to avoid overflow
    l = l.astype(np.float32)
    
    # Enhance exposure by scaling the L channel
    l *= exposure_factor
    
    # Clip the values to [0, 255] and convert back to uint8
    l = np.clip(l, 0, 255).astype(np.uint8)
    
    # Merge the channels back
    enhanced_lab = cv2.merge((l, a, b))
    
    # Convert back to BGR color space
    enhanced_image = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)
    return enhanced_image

# Load the image
image_path = r'C:\Users\User\OneDrive\Documents\Data Challenge 3\DC-3-Group-12\Preprocessing\7393_F1_f000513.jpg'
image = cv2.imread(image_path)

if image is None:
    print("Error: Could not load image. Check the file path.")
else:
    # Enhance the exposure of the image
    enhanced_image = enhance_exposure(image, exposure_factor=1.2)

    # Display the original and enhanced images
    cv2.imshow('Original Image', image)
    cv2.imshow('Enhanced Exposure Image', enhanced_image)

    # Wait for a key press and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()


This code changes hue. Hue can be thought of as the “shade” of the colors in an image. The Hue augmentation changes the color channels of an input image at random, causing a model to explore several color schemes for objects and scenes in the image. This strategy is important for ensuring that a model does not memorize the colors of a given object or scene.

In [7]:
import cv2
import numpy as np

def enhance_hue(image, hue_shift=30):
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Split the channels
    h, s, v = cv2.split(hsv_image)
    
    # Enhance the hue by adding the shift value
    h = h.astype(np.int32)  # Convert to int32 to avoid overflow
    h = (h + hue_shift) % 180  # Shift hue and wrap around using modulo operation
    h = h.astype(np.uint8)  # Convert back to uint8
    
    # Merge the channels back
    enhanced_hsv = cv2.merge([h, s, v])
    
    # Convert back to BGR color space
    enhanced_image = cv2.cvtColor(enhanced_hsv, cv2.COLOR_HSV2BGR)
    return enhanced_image

# Load the image
image_path = r'C:\Users\User\OneDrive\Documents\Data Challenge 3\DC-3-Group-12\Preprocessing\7393_F1_f000513.jpg'
image = cv2.imread(image_path)

if image is None:
    print("Error: Could not load image. Check the file path.")
else:
    # Enhance the hue of the image
    enhanced_image = enhance_hue(image, hue_shift=30)

    # Display the original and enhanced images
    cv2.imshow('Original Image', image)
    cv2.imshow('Enhanced Hue Image', enhanced_image)

    # Wait for a key press and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [5]:
pip install opencv-python


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/38.8 MB 5.1 MB/s eta 0:00:08
    --------------------------------------- 0.6/38.8 MB 6.2 MB/s eta 0:00:07
    --------------------------------------- 0.9/38.8 MB 6.1 MB/s eta 0:00:07
   - -------------------------------------- 1.6/38.8 MB 8.1 MB/s eta 0:00:05
   -- ------------------------------------- 2.1/38.8 MB 9.1 MB/s eta 0:00:05
   -- ------------------------------------- 2.6/38.8 MB 8.9 MB/s eta 0:00:05
   --- ------------------------------------ 3.1/38.8 MB 9.4 MB/s eta 0:00:04
   --- ------------------------------------ 3.6/38.8 MB 9.2 MB/s eta 0:00:04
   ---- ----------------------------------- 4.2/38.8 MB 10.0 MB/s eta 0:00:04
   ----- ---------------------------------- 4.9/38.8 MB 10.5 MB/s eta 0:00:04
   ----- ---------------------------------- 5.5/38.8 MB 10.6 MB/s eta 0:00:04
   ------ 


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


These dont do anything 

In [21]:
import cv2
import numpy as np

def adjust_brightness_contrast(image, alpha=1.2, beta=10):
    # Adjust brightness and contrast with clipping
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image

def apply_clahe(image):
    # Convert image to LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_image)

    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)

    # Merge the CLAHE-enhanced L channel back with A and B channels
    enhanced_image = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(enhanced_image, cv2.COLOR_LAB2BGR)

def apply_denoising(image):
    # Apply fast NlMeansDenoisingColored
    denoised_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    return denoised_image

def selective_contrast(image):
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Selectively increase contrast in darker areas
    l_mask = l < 120  # Mask for darker areas
    l = np.where(l_mask, l * 1.2, l)  # Increase contrast in masked areas
    lab_adjusted = cv2.merge([l, a, b])

    # Convert back to BGR color space
    result = cv2.cvtColor(lab_adjusted, cv2.COLOR_LAB2BGR)
    return result

# Load the uploaded image
image_path = r'C:\Users\User\OneDrive\Documents\Data Challenge 3\DC-3-Group-12\Preprocessing\7393_F1_f000513.jpg'
image = cv2.imread(image_path)

if image is None:
    print("Error: Could not load image. Check the file path.")
else:
    # Step 1: Apply CLAHE for better contrast
    clahe_image = apply_clahe(image)

    # Step 2: Selective contrast enhancement
    selective_image = selective_contrast(clahe_image)

    # Step 3: Apply slight brightness/contrast adjustment
    adjusted_image = adjust_brightness_contrast(selective_image, alpha=1.1, beta=20)

    # Step 4: Apply denoising to smooth the image
    final_image = apply_denoising(adjusted_image)

    # Display the original and enhanced images side by side
    cv2.imshow('Original Image', image)
    cv2.imshow('Enhanced Image', final_image)

    # Wait for a key press and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [22]:
import cv2
import numpy as np

def adjust_brightness_contrast(image, alpha=1.2, beta=10):
    # Adjust brightness and contrast with clipping
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image

def apply_clahe(image):
    # Convert image to LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_image)

    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)

    # Merge the CLAHE-enhanced L channel back with A and B channels
    enhanced_image = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(enhanced_image, cv2.COLOR_LAB2BGR)

def apply_denoising(image):
    # Apply fast NlMeansDenoisingColored
    denoised_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    return denoised_image

def selective_contrast(image):
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Ensure l is of type float32 for proper processing
    l = l.astype(np.float32)

    # Selectively increase contrast in darker areas
    l_mask = l < 120  # Mask for darker areas
    l = np.where(l_mask, l * 1.2, l)  # Increase contrast in masked areas

    # Clip values to ensure they stay within the 0-255 range
    l = np.clip(l, 0, 255).astype(np.uint8)  # Convert back to uint8 for merging

    # Merge the channels back
    lab_adjusted = cv2.merge([l, a, b])

    # Convert back to BGR color space
    result = cv2.cvtColor(lab_adjusted, cv2.COLOR_LAB2BGR)
    return result

# Load the uploaded image
image_path = r'C:\Users\User\OneDrive\Documents\Data Challenge 3\DC-3-Group-12\Preprocessing\7393_F1_f000513.jpg'
image = cv2.imread(image_path)

if image is None:
    print("Error: Could not load image. Check the file path.")
else:
    # Step 1: Apply CLAHE for better contrast
    clahe_image = apply_clahe(image)

    # Step 2: Selective contrast enhancement
    selective_image = selective_contrast(clahe_image)

    # Step 3: Apply slight brightness/contrast adjustment
    adjusted_image = adjust_brightness_contrast(selective_image, alpha=1.1, beta=20)

    # Step 4: Apply denoising to smooth the image
    final_image = apply_denoising(adjusted_image)

    # Display the original and enhanced images side by side
    cv2.imshow('Original Image', image)
    cv2.imshow('Enhanced Image', final_image)

    # Wait for a key press and close the windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()
